# Colab demo: Edit rubric.json and re-run scoring

This notebook demonstrates how to change `config/rubric.json` in a Colab session and re-run `calculate_sustainability_scores` to observe the effect on scores. Run the cells in order.

## 1 — Setup (clone or skip if already in repo)

If you opened this notebook from GitHub or uploaded the repository files into Colab, skip the clone step. Otherwise, run the clone cell to get the repo into the Colab session. Replace `<YOUR_GITHUB_REPO_URL>` with your repo URL.

In [ ]:
# Clone the repo into the Colab session (uncomment and set your repo URL if needed)
# !git clone <YOUR_GITHUB_REPO_URL> himcm_repo
# %cd himcm_repo

# Install dependencies (run once)
!pip install -r requirements.txt

In [ ]:
# 2 — Inspect or create the rubric file
import json, os
from pathlib import Path
p = Path('config') / 'rubric.json'
if not p.exists():
    print('rubric.json not found — creating a default one at', p)
    p.parent.mkdir(parents=True, exist_ok=True)
    default = {
        "Stadium_Type": {"dome": 100, "open-air": 50, "default": 50},
        "Stadium_leed_cert": {"platinum": 100, "gold": 80, "silver": 60, "default": 30},
        "Waste_Management": {"recycling only": 70, "recycling and composting": 90, "default": 50},
        "Future_Developments": {"planned_green_projects": 80, "no_known_plans": 30, "default": 50}
    }
    p.write_text(json.dumps(default, indent=2))

print('Current rubric:')
print(p)
print('
---')
print(p.read_text())

In [ ]:
# 3 — Create a small synthetic DataFrame and run the scoring (before change)
import pandas as pd
from pprint import pprint
df = pd.DataFrame([
    {
        'City': 'A City', 'State': 'AA', 'Times_Hosted': 0, 'Last_Hosted_Year': 1900,
        'Stadium_Type': 'open-air', 'Stadium_leed_cert': 'Gold Certified', 'Avg_Feb_Temp_F': 45,
        'Alltransit_Score': 70, 'Renewable_Energy_Pct': 20, 'Waste_Diversion_Pct': 30,
        'Green_Legacy_Projects': 1, 'Carbon_Offset_Programs': 0, 'Waste_Management': 'recycling only', 'Future_Developments': 'no_known_plans'
    },
    {
        'City': 'B Town', 'State': 'BB', 'Times_Hosted': 2, 'Last_Hosted_Year': 2018,
        'Stadium_Type': 'dome', 'Stadium_leed_cert': 'Platinum', 'Avg_Feb_Temp_F': 30,
        'Alltransit_Score': 85, 'Renewable_Energy_Pct': 75, 'Waste_Diversion_Pct': 80,
        'Green_Legacy_Projects': 5, 'Carbon_Offset_Programs': 2, 'Waste_Management': 'recycling and composting', 'Future_Developments': 'planned_green_projects'
    },
    {
        'City': 'C Ville', 'State': 'CC', 'Times_Hosted': 1, 'Last_Hosted_Year': 2005,
        'Stadium_Type': 'indoor', 'Stadium_leed_cert': 'Silver', 'Avg_Feb_Temp_F': 40,
        'Alltransit_Score': 60, 'Renewable_Energy_Pct': 35, 'Waste_Diversion_Pct': 50,
        'Green_Legacy_Projects': 2, 'Carbon_Offset_Programs': 1, 'Waste_Management': 'minimal', 'Future_Developments': 'no_known_plans'
    },
    {
        'City': 'D City', 'State': 'DD', 'Times_Hosted': 0, 'Last_Hosted_Year': 1990,
        'Stadium_Type': 'retractable roof', 'Stadium_leed_cert': null, 'Avg_Feb_Temp_F': 55,
        'Alltransit_Score': 40, 'Renewable_Energy_Pct': 10, 'Waste_Diversion_Pct': 10,
        'Green_Legacy_Projects': 0, 'Carbon_Offset_Programs': 0, 'Waste_Management': 'no_program', 'Future_Developments': 'major_green_investments'
    }
])

# Import scoring function — ensure repo is on path if needed
import sys, os
if os.getcwd() not in sys.path:
    sys.path.insert(0, os.getcwd())
from himcm.core import calculate_sustainability_scores

df_before = calculate_sustainability_scores(df.copy())
print('Before change — scaled Z:')
print(df_before[['City','Sustainability_Z_Scaled','Raw_Score']])

In [ ]:
# 4 — Edit the rubric in-place: lower the score for 'gold' and reduce 'recycling only'
import json
p = Path('config') / 'rubric.json'
r = json.loads(p.read_text())
print('Original stadium_leed_cert.gold =', r.get('Stadium_leed_cert', {}).get('gold'))
print('Original Waste_Management.recycling only =', r.get('Waste_Management', {}).get('recycling only'))
# Make edits
r.setdefault('Stadium_leed_cert', {})['gold'] = 40
r.setdefault('Waste_Management', {})['recycling only'] = 30
# Write back
p.write_text(json.dumps(r, indent=2))
print('
Wrote modified rubric:')
print(p.read_text())

In [ ]:
# 5 — Re-run scoring after rubric change and compare
df_after = calculate_sustainability_scores(df.copy())
compare = df_before.set_index('City')[['Raw_Score','Sustainability_Z_Scaled']].join(
    df_after.set_index('City')[['Raw_Score','Sustainability_Z_Scaled']], lsuffix='_before', rsuffix='_after'
)
print('Comparison (before vs after):')
print(compare)

### Notes
- The notebook edits `config/rubric.json` in-place. In a production workflow you may prefer to keep version-controlled rubrics in Git and create separate scenario files for experiments.
- If you clone the repo in Colab, the changes persist only within the Colab session unless you push them back to your Git repo or save them to Google Drive.